In [ ]:
%pip install olll

In [ ]:
import math
from decimal import *
getcontext().prec = 200
import olll

In [ ]:
PRIMES = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103]
c = 1350995397927355657956786955603012410260017344805998076702828160316695004588429433

Mathematically the cipher is calculated as:
$$c = \lfloor \sum_i{m_i \left({16}^{64}\sqrt{p_i}\right)}\rfloor$$
This means the cipher is a linear combination (with integer coefficients) of numbers - sounds pretty like a lattic problem.
Later I found this lecture summary: https://people.csail.mit.edu/vinodv/6876-Fall2015/L5.pdf
In this lecture a method to solve this kind of probelm is presented, it involves transforming this problem to shortest vector problem (SVP) and then using LLL algorithm to solve it.
By defining:
$$ M=	\begin{pmatrix}
1 & 0 & 0 & ... & 0 & \left[{16}^{64}\sqrt{p_0}\right]\\
0 & 1 & 0 & ... & 0 & \left[{16}^{64}\sqrt{p_1}\right]\\
  &   &  &  ...  &  &\\
0 & 0 & 0 & ... & 1 & \left[{16}^{64}\sqrt{p_{N-1}}\right]\\
0 & 0 & 0 & ... & 0 & c\\
\end{pmatrix}$$
This definition is great as we know there must be a linear combination of the first $N$ which will give us $c$ in the last column and could cancel it. This will shorten the vector drastically, and we can use LLL to find it.

In [ ]:
encrypt_basis = [round(Decimal(PRIMES[i]).sqrt()*16**64) for i in range(len(PRIMES))]
m = [[1 if j==i else 0 for j in range(len(encrypt_basis))] + [encrypt_basis[i]] for i in range(len(encrypt_basis))]
m.append([0]*len(encrypt_basis) + [c])
m

As I couldn't find any implementation of LLL in pure python, I used sagemath to solve this problem. The code is attached below (you can run it on https://sagecell.sagemath.org/):
```python
m = ... # matrix generated above
m = Matrix(ZZ, m)
lll_result = m.LLL()
print("".join([chr(-x) for x in lll_result[0][:-1]])) #convert first row of LLL to string
```